In [1]:
import numpy as np

from bokeh.io import output_notebook, show
import bokeh.plotting

import scipy.integrate
import scipy.optimize

import matplotlib.pyplot as plt

import colorcet

interactive_python_plots = True
import os
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "1pbk8bdv6u5psm9k9u3o38d9iohudpemj63bekpdcrbnefl5la44"

# Optoepressilator - constant light intensity

In [13]:
# Show Bokeh output inline
output_notebook()

def protein_repressilator_rhs(x, t, beta, beta2, alpha, n):
    """
    Returns 3-array of (dx_1/dt, dx_2/dt, dx_3/dt)
    """
    x_1, x_2, x_3, xp = x

    return np.array(
        [
            beta / (1 + x_3 ** n) - alpha*x_1,
            beta / (1 + (x_1 + xp) ** n) - alpha*x_2,
            beta / (1 + x_2 ** n) - alpha*x_3,
            beta2 - alpha*xp
        ]
    )


# Initial condiations
x0 = np.array([1, 1.1, 1.2, 0])

# Number of points to use in plots
n_points = 1000

# Widgets for controlling parameters
beta_slider_protein = bokeh.models.Slider(title="β (production of x,y,z)", start=0, end=1000, step=1, value=10)
beta2_slider_protein = bokeh.models.Slider(title="β' (production of x')", start=0, end=5, step=0.1, value=1)
alpha_slider_protein = bokeh.models.Slider(title="α (universal decay)", start=0, end=3, step=0.1, value=1)
n_slider_protein = bokeh.models.Slider(title="n (Hill)", start=1, end=5, step=0.1, value=3)

# Solve for species concentrations
def _solve_protein_repressilator(beta, beta2, alpha, n, t_max):
    t = np.linspace(0, t_max, n_points)
    x = scipy.integrate.odeint(protein_repressilator_rhs, x0, t, args=(beta, beta2, alpha, n))

    return t, x.transpose()


# Obtain solution for plot
t, x = _solve_protein_repressilator(beta_slider_protein.value, beta2_slider_protein.value, alpha_slider_protein.value, n_slider_protein.value, 100.0)

# Build the plot
colors = colorcet.b_glasbey_category10[:4]

p_rep = bokeh.plotting.figure(
    frame_width=550, frame_height=200, x_axis_label="t", x_range=[0, 100.0]
)

cds = bokeh.models.ColumnDataSource(data=dict(t=t, x1=x[0], x2=x[1], x3=x[2], xp=x[3]))
labels = dict(x1="x", x2="y", x3="z", xp="x'")
for color, x_val in zip(colors, labels):
    p_rep.line(
        source=cds,
        x="t",
        y=x_val,
        color=color,
        legend_label=labels[x_val],
        line_width=2,
    )

p_rep.legend.location = "bottom_right"


# Set up callbacks
def _callback(attr, old, new):
    t, x = _solve_protein_repressilator(beta_slider_protein.value, beta2_slider_protein.value, alpha_slider_protein.value, n_slider_protein.value, p_rep.x_range.end)
    cds.data = dict(t=t, x1=x[0], x2=x[1], x3=x[2], xp=x[3])


beta_slider_protein.on_change("value", _callback)
beta2_slider_protein.on_change("value", _callback)
alpha_slider_protein.on_change("value", _callback)
n_slider_protein.on_change("value", _callback)
p_rep.x_range.on_change("end", _callback)

# Build layout
protein_repressilator_layout = bokeh.layouts.column(
    p_rep,
    bokeh.layouts.Spacer(height=10),
    bokeh.layouts.row(
        bokeh.layouts.Spacer(width=70),
        bokeh.layouts.column(beta_slider_protein, beta2_slider_protein, alpha_slider_protein, n_slider_protein,width=150),
    ),
)

# Build the app
def protein_repressilator_app(doc):
    doc.add_root(protein_repressilator_layout)


show(protein_repressilator_app)

Loading BokehJS ...

In [15]:
# Show Bokeh output inline
output_notebook()

def beta2_function(t, period, pulse_duration, pulse_intensity):
    """
    Returns beta2 as a function of time
    t: current time
    period: time between pulses
    pulse_duration: duration of each pulse
    pulse_intensity: intensity of the pulse (beta2 value during the pulse)
    """
    if t % period < pulse_duration:
        return pulse_intensity
    else:
        return 0

def protein_repressilator_rhs(x, t, beta, beta2_func, alpha, n):
    """
    Returns 4-array of (dx_1/dt, dx_2/dt, dx_3/dt, dx_p/dt)
    """
    x_1, x_2, x_3, xp = x

    return np.array(
        [
            beta / (1 + x_3 ** n) - alpha*x_1,
            beta / (1 + (x_1 + xp) ** n) - alpha*x_2,
            beta / (1 + x_2 ** n) - alpha*x_3,
            beta2_func(t) - alpha*xp
        ]
    )

# Initial conditions
x0 = np.array([1, 1.1, 1.2, 0])

# Number of points to use in plots
n_points = 1000

# Widgets for controlling parameters
beta_slider_protein = bokeh.models.Slider(title="β (production of x,y,z)", start=0, end=25, step=1, value=10)
alpha_slider_protein = bokeh.models.Slider(title="α (universal decay)", start=0, end=3, step=0.1, value=1)
n_slider_protein = bokeh.models.Slider(title="n (Hill)", start=1, end=5, step=0.1, value=3)
period_slider = bokeh.models.Slider(title="Period between pulses", start=1, end=20, step=1, value=10)
pulse_duration_slider = bokeh.models.Slider(title="Pulse duration", start=0.1, end=5, step=0.1, value=2)
pulse_intensity_slider = bokeh.models.Slider(title="Pulse intensity (β')", start=0, end=5, step=0.1, value=3)

# Solve for species concentrations
def _solve_protein_repressilator(beta, alpha, n, period, pulse_duration, pulse_intensity, t_max):
    t = np.linspace(0, t_max, n_points)
    beta2_func = lambda t: beta2_function(t, period, pulse_duration, pulse_intensity)
    x = scipy.integrate.odeint(protein_repressilator_rhs, x0, t, args=(beta, beta2_func, alpha, n))

    return t, x.transpose()


# Obtain solution for plot
# Obtain solution for plot
t, x = _solve_protein_repressilator(
    beta_slider_protein.value,
    alpha_slider_protein.value,
    n_slider_protein.value,
    period_slider.value,
    pulse_duration_slider.value,
    pulse_intensity_slider.value,
    100.0
)
# Build the plot
colors = colorcet.b_glasbey_category10[:4]

p_rep = bokeh.plotting.figure(
    frame_width=550, frame_height=200, x_axis_label="t", x_range=[0, 100.0]
)

cds = bokeh.models.ColumnDataSource(data=dict(t=t, x1=x[0], x2=x[1], x3=x[2], xp=x[3]))
labels = dict(x1="x", x2="y", x3="z", xp="x'")
for color, x_val in zip(colors, labels):
    p_rep.line(
        source=cds,
        x="t",
        y=x_val,
        color=color,
        legend_label=labels[x_val],
        line_width=2,
    )

p_rep.legend.location = "bottom_right"


# Set up callbacks
def _callback(attr, old, new):
    t, x = _solve_protein_repressilator(
        beta_slider_protein.value,
        alpha_slider_protein.value,
        n_slider_protein.value,
        period_slider.value,
        pulse_duration_slider.value,
        pulse_intensity_slider.value,
        p_rep.x_range.end
    )
    cds.data = dict(t=t, x1=x[0], x2=x[1], x3=x[2], xp=x[3])


beta_slider_protein.on_change("value", _callback)
# beta2_slider_protein.on_change("value", _callback)
alpha_slider_protein.on_change("value", _callback)
n_slider_protein.on_change("value", _callback)

period_slider.on_change("value", _callback)
pulse_duration_slider.on_change("value", _callback)
pulse_intensity_slider.on_change("value", _callback)

p_rep.x_range.on_change("end", _callback)

# Build layout
protein_repressilator_layout = bokeh.layouts.column(
    p_rep,
    bokeh.layouts.Spacer(height=10),
    bokeh.layouts.row(
        bokeh.layouts.Spacer(width=70),
        bokeh.layouts.column(beta_slider_protein, alpha_slider_protein, n_slider_protein, period_slider, pulse_duration_slider, pulse_intensity_slider, width=150),
    ),
)

# Build the app
def protein_repressilator_app(doc):
    doc.add_root(protein_repressilator_layout)


show(protein_repressilator_app)

Loading BokehJS ...